In [2]:
import sagemaker
import pandas as pd
from time import gmtime, strftime
import time
from datetime import datetime

print('Start', datetime.now().strftime("%H:%M:%S"))

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()

Start 07:43:39


In [ ]:
import logging
logging.getLogger('sagemaker').setLevel(logging.CRITICAL)

In [3]:
from sagemaker.amazon.amazon_estimator import get_image_uri

training_image = get_image_uri(sagemaker_session.boto_region_name, 'xgboost', repo_version="0.90-1")
print(training_image)

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:0.90-1-cpu-py3


In [16]:
bucket = 'datalake-published-data-907317471167-us-east-1-qkg9331'
prefix = 'cmaps-ml2'
train_part, val_part = ('split=train', 'split=validation')
output_prefix = 'output'

# Training the XGBoost model¶

In [24]:
timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
s3_train_data = f's3://{bucket}/{prefix}/{train_part}'
s3_validation_data = f's3://{bucket}/{prefix}/{val_part}'
s3_output_location = f's3://{bucket}/{output_prefix}/xgboost_model'

In [20]:
xgb_model = sagemaker.estimator.Estimator(training_image,
                                          role, 
                                          train_instance_count=1, 
                                          train_instance_type='ml.m4.xlarge',
                                          train_volume_size = 20,
                                          train_max_run = 3600,
                                          input_mode= 'Pipe',
                                          output_path=s3_output_location,
                                          sagemaker_session=sagemaker_session)

xgb_model.set_hyperparameters(objective = "reg:squarederror",
                              eta = .2,
                              gamma = 4,
                              max_depth = 5,
                              num_round = 50,
                              subsample = 0.7,
                              silent = 0,
                              min_child_weight = 6)

train_data = sagemaker.session.s3_input(s3_train_data, distribution='FullyReplicated', compression='Gzip',
                        content_type='text/csv', s3_data_type='S3Prefix')
validation_data = sagemaker.session.s3_input(s3_validation_data, distribution='FullyReplicated', compression='Gzip', 
                             content_type='text/csv', s3_data_type='S3Prefix')

data_channels = {'train': train_data, 'validation': validation_data}

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_run has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_volume_size has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [21]:
xgb_model.fit(inputs=data_channels, logs=True)

2020-12-01 08:33:03 Starting - Starting the training job...
2020-12-01 08:33:05 Starting - Launching requested ML instances.........
2020-12-01 08:34:36 Starting - Preparing the instances for training......
2020-12-01 08:35:52 Downloading - Downloading input data...
2020-12-01 08:36:18 Training - Downloading the training image..INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value reg:squarederror to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Pipe path /opt/ml/input/data/train found.
INFO:root:Pipe path /opt/ml/input/data/validation found.
INFO:root:Single node training.
INFO:root:Train matrix has 14023 rows
INFO:root:Validation matrix has 6608 rows
[0]#011train-rmse:104.644#011validation-rmse:103.088
[1]#011train-rmse:86.2865#011

# Building an inference Pipeline using SageMaker SDK

In [22]:
# XGBoost model artifacts 
xgb_model.model_data

's3://datalake-published-data-907317471167-us-east-1-qkg9331/output/xgboost_model/sagemaker-xgboost-2020-12-01-08-33-03-814/output/model.tar.gz'

In [26]:
endpoint_name = 'inference-pipeline-ep-' + timestamp_prefix

In [ ]:
predictor = xgb_model.deploy(initial_instance_count=1, instance_type='ml.c4.xlarge', endpoint_name=endpoint_name)

-----

In [75]:
from sagemaker.predictor import json_serializer, csv_serializer, json_deserializer, RealTimePredictor

In [80]:
predictor = RealTimePredictor(endpoint_name=endpoint_name, sagemaker_session=sagemaker_session, serializer=csv_serializer,
                                content_type='text/csv', accept='text/csv')

The class RealTimePredictor has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
content_type is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
accept is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [91]:
with open('aws-bb-cmapss/data/train_FD001.txt') as csvf:
    for i, row in enumerate(csvf.readlines()):
        prediction = predictor.predict(','.join(row.strip().split(' ')[1:]))
        print(prediction)
        if i > 100:
            break


b'208.28956604003906'
b'202.65234375'
b'197.8765411376953'
b'217.3982391357422'
b'204.9069366455078'
b'208.5284423828125'
b'195.82093811035156'
b'187.29452514648438'
b'199.78073120117188'
b'186.3865509033203'
b'203.58816528320312'
b'212.28073120117188'
b'185.60752868652344'
b'197.88758850097656'
b'187.69908142089844'
b'193.71083068847656'
b'204.13967895507812'
b'199.91458129882812'
b'180.61285400390625'
b'176.0583953857422'
b'200.1953582763672'
b'180.85430908203125'
b'176.80828857421875'
b'179.3096466064453'
b'193.01251220703125'
b'175.11705017089844'
b'185.15428161621094'
b'188.4100341796875'
b'191.44964599609375'
b'186.1831817626953'
b'182.75543212890625'
b'178.76129150390625'
b'174.25015258789062'
b'173.32997131347656'
b'184.7650909423828'
b'186.24107360839844'
b'192.09677124023438'
b'190.40821838378906'
b'154.4621124267578'
b'172.5054931640625'
b'173.3092041015625'
b'158.70838928222656'
b'165.2967071533203'
b'170.06790161132812'
b'159.53082275390625'
b'155.09596252441406'
b'165.775

In [71]:
predictor.delete_endpoint()